# Imports & Setup

In [1]:
# pybotics imports
from pybotics.robot_model import UR10

# arrays, vectors, and matrices
import numpy as np
np.set_printoptions(suppress=True, precision=3)

# Computation Speed

In [52]:
robot = UR10()

q = np.zeros(robot.num_dof)
q[0] = np.deg2rad(1)
print('Desired joints: {}'.format(q))

q_seed = np.zeros(robot.num_dof)
print('Seed joints: {}'.format(q_seed))

pose = robot.fk(q)
print('Desired pose:\n{}'.format(pose))

Desired joints: [0.017 0.    0.    0.    0.    0.   ]
Seed joints: [0. 0. 0. 0. 0. 0.]
Desired pose:
[[    1.        0.        0.017 -1179.65 ]
 [    0.017     0.       -1.     -276.73 ]
 [   -0.        1.        0.        2.3  ]
 [    0.        0.        0.        1.   ]]


## Built-in Method

In [116]:
# very slow for 6-axis robot...
q_actual = robot.ik(pose, q=q_seed, max_iter=10, alpha=10)

In [118]:
print('Joints diff:\n{}'.format(q-q_actual))
print('Pose diff:\n{}'.format(pose-robot.fk(q_actual)))

Joints diff:
[-0.733  1.887  2.761 -1.926  0.217 -1.27 ]
Pose diff:
[[    0.952    -0.851    -0.505 -1471.529]
 [   -0.114    -0.513    -0.152  -201.665]
 [    0.99      0.89      0.088  -225.107]
 [    0.        0.        0.        0.   ]]


## SciPy Optimization

In [78]:
from scipy.optimize import leastsq, minimize
from pybotics.geometry import matrix_2_euler_zyx
import numpy as np

def fitness_func(q, robot, pose):
    desired_vector = matrix_2_euler_zyx(pose)
    
    current_pose = robot.fk(q)
    current_vector = matrix_2_euler_zyx(current_pose)
    
    vector_diff = desired_vector - current_vector
    return vector_diff

def fitness_func_scalar(q, robot, pose):
    x = fitness_func(q, robot, pose)
    return np.sum(np.square(x))

### `leastsq`

In [93]:
q_actual = leastsq(fitness_func, q_seed, args=(robot, pose))
q_actual

(array([ 0.017,  0.   ,  0.   ,  0.   ,  0.   , -0.   ]), 2)

In [105]:
print('Joints diff:\n{}'.format(q-q_actual.x))
print('Pose diff:\n{}'.format(pose-robot.fk(q_actual.x)))

Joints diff:
[ 0.  0. -0.  0.  0. -0.]
Pose diff:
[[ 0. -0. -0. -0.]
 [-0.  0. -0. -0.]
 [ 0.  0.  0.  0.]
 [ 0.  0.  0.  0.]]


In [62]:
%%timeit -n 5 -r 5
leastsq(fitness_func, q_seed, args=(robot, pose))

18.1 ms ± 1.51 ms per loop (mean ± std. dev. of 5 runs, 5 loops each)


### `minimize(method=’Nelder-Mead’)`

In [97]:
q_actual = minimize(fitness_func_scalar, q_seed, args=(robot, pose), method='Nelder-Mead')
q_actual

 final_simplex: (array([[ 0.017,  0.001, -0.002,  0.004,  0.004, -0.014],
       [ 0.017,  0.001, -0.002,  0.004,  0.004, -0.014],
       [ 0.017,  0.001, -0.002,  0.004,  0.004, -0.014],
       [ 0.017,  0.001, -0.002,  0.004,  0.004, -0.014],
       [ 0.017,  0.001, -0.002,  0.004,  0.004, -0.014],
       [ 0.017,  0.001, -0.002,  0.004,  0.004, -0.014],
       [ 0.017,  0.001, -0.002,  0.004,  0.004, -0.014]]), array([0., 0., 0., 0., 0., 0., 0.]))
           fun: 0.00018137016527255886
       message: 'Optimization terminated successfully.'
          nfev: 256
           nit: 151
        status: 0
       success: True
             x: array([ 0.017,  0.001, -0.002,  0.004,  0.004, -0.014])

In [104]:
print('Joints diff:\n{}'.format(q-q_actual.x))
print('Pose diff:\n{}'.format(pose-robot.fk(q_actual.x)))

Joints diff:
[ 0.  0. -0.  0.  0. -0.]
Pose diff:
[[ 0. -0. -0. -0.]
 [-0.  0. -0. -0.]
 [ 0.  0.  0.  0.]
 [ 0.  0.  0.  0.]]


In [112]:
%%timeit -n 5 -r 5
minimize(fitness_func_scalar, q_seed, args=(robot, pose), method='Nelder-Mead')

122 ms ± 15.9 ms per loop (mean ± std. dev. of 5 runs, 5 loops each)


### `minimize(method=’Powell’)`

In [99]:
q_actual = minimize(fitness_func_scalar, q_seed, args=(robot, pose), method='Powell')
q_actual

   direc: array([[ 0.,  0.,  0.,  0.,  0.,  1.],
       [-0., -0.,  0.,  0.,  0., -0.],
       [ 0.,  0.,  1.,  0.,  0.,  0.],
       [ 0.,  0.,  0.,  1.,  0.,  0.],
       [ 0.,  0.,  0.,  0.,  1.,  0.],
       [ 0.,  0., -0., -0.,  0.,  0.]])
     fun: 7.231922700377718e-17
 message: 'Optimization terminated successfully.'
    nfev: 622
     nit: 8
  status: 0
 success: True
       x: array([ 0.017, -0.   ,  0.   , -0.   , -0.   ,  0.   ])

In [103]:
print('Joints diff:\n{}'.format(q-q_actual.x))
print('Pose diff:\n{}'.format(pose-robot.fk(q_actual.x)))

Joints diff:
[ 0.  0. -0.  0.  0. -0.]
Pose diff:
[[ 0. -0. -0. -0.]
 [-0.  0. -0. -0.]
 [ 0.  0.  0.  0.]
 [ 0.  0.  0.  0.]]


In [113]:
%%timeit -n 5 -r 5
minimize(fitness_func_scalar, q_seed, args=(robot, pose), method='Powell')

255 ms ± 3.54 ms per loop (mean ± std. dev. of 5 runs, 5 loops each)


### `minimize(method=’L-BFGS-B’)`

In [106]:
q_actual = minimize(fitness_func_scalar, q_seed, args=(robot, pose), method='L-BFGS-B')
q_actual

      fun: 3.135500321451823e-11
 hess_inv: <6x6 LbfgsInvHessProduct with dtype=float64>
      jac: array([ 0.004,  0.008,  0.   ,  0.   , -0.   ,  0.   ])
  message: b'CONVERGENCE: REL_REDUCTION_OF_F_<=_FACTR*EPSMCH'
     nfev: 203
      nit: 13
   status: 0
  success: True
        x: array([ 0.017, -0.   ,  0.   ,  0.   ,  0.   , -0.   ])

In [107]:
print('Joints diff:\n{}'.format(q-q_actual.x))
print('Pose diff:\n{}'.format(pose-robot.fk(q_actual.x)))

Joints diff:
[ 0.  0. -0. -0. -0.  0.]
Pose diff:
[[-0.  0.  0. -0.]
 [ 0.  0.  0. -0.]
 [-0.  0.  0. -0.]
 [ 0.  0.  0.  0.]]


In [114]:
%%timeit -n 5 -r 5
minimize(fitness_func_scalar, q_seed, args=(robot, pose), method='L-BFGS-B')

81.7 ms ± 3.35 ms per loop (mean ± std. dev. of 5 runs, 5 loops each)
